Разделяем слой с остановками на уникальные маршруты, поскольку есть маршруты в разных областях ЛО c одинаковыми названиями, необходимо применить кластеризацию по расположению с искуственным шумом. Если все маршруты уникальные это не требуется

In [1]:
from shapely import Point
import pandas as pd
from tqdm.auto import tqdm
import geopandas as gpd
from dongraphio.utils.geometry_utils import clusterize_kmeans_geo_points

tqdm.pandas()

stops_gdf = gpd.read_file("stopsLO.geojson")
stops_gdf.rename(columns={"Автоб": "route"}, inplace=True)
stops_gdf.dropna(subset="route", axis=0, inplace=True, how="any")
unique_routes = set()
for index, row in (stops_gdf.iterrows()):
    for i in str(row["route"]).replace('.', ',').rstrip().lstrip().split(","):
        if i != "":
            unique_routes.add(i.lstrip().rstrip())
unique_routes = pd.DataFrame(index=list(unique_routes))
unique_routes["geometry"] = [[]] * len(unique_routes)
for index, row in (stops_gdf.iterrows()):
    for i in str(row["route"]).replace('.', ',').rstrip().lstrip().split(","):
        if i != "":
            unique_routes.loc[i.lstrip().rstrip(), "geometry"] = unique_routes.loc[i.lstrip().rstrip()]["geometry"] + [
                row["geometry"]]

devided_routes = pd.DataFrame()
noise_points = [[31.21239, 58.61124], [31.22239, 58.62161], [31.23239, 58.61171], [31.24239, 58.62129],
                [31.23239, 58.64131],
                [32.92133, 59.02899], [32.94933, 59.06599], [32.93943, 59.03599], [32.92733, 59.08999],
                [32.92333, 59.00599],
                [35.58878, 59.48945], [35.58933, 59.48359], [35.58643, 59.489759], [35.58879, 59.48688],
                [35.58869, 59.485749]]
noise_points = [Point(x) for x in noise_points]
for index, row in tqdm(unique_routes.iterrows(), total=unique_routes.shape[0]):
    clusterized = clusterize_kmeans_geo_points(row, noise_points)
    if clusterized.shape[0] > 1:
        clusterized.index = index + "." + clusterized.index.astype(str)
        devided_routes = pd.concat([devided_routes, clusterized])
    else:
        clusterized.index = [index]
        devided_routes = pd.concat([devided_routes, clusterized])

  0%|          | 0/593 [00:00<?, ?it/s]

In [2]:
from src.dongraphio.utils import get_osmnx_graph
import networkx as nx

import osmnx as ox

drive_graph: nx.MultiDiGraph = get_osmnx_graph(
    176095, 32636, "drive_service", truncate_by_edge=True
)
# from shapely import LineString
# for i in drive_graph.edges(data=True):
#     i[2]['geometry'] = from_wkt(i[2]['geometry'])
# 
# gdf_graph_edges = ox.graph_to_gdfs(drive_graph, nodes=False)
# gdf_graph_edges.to_file('graph_LO.geojson')

2024-03-30 20:26:37.017 | DEBUG    | src.dongraphio.utils.graphs:get_osmnx_graph:58 - Extracting and preparing drive_service graph from OSM ...
C:\Users\OEM\AppData\Local\pypoetry\Cache\virtualenvs\dongraphio-ppw38GG8-py3.10\lib\site-packages\osmnx\_overpass.py:211: UserWarning: This area is 51 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [3]:

from shapely import from_wkt
import networkx as nx
from src.dongraphio.utils.graph_utils import buffer_directed_roads, nx_to_gdf

for i in drive_graph.edges(data=True):
    i[2]['geometry'] = from_wkt(str(i[2]['geometry']))
drive_graph_gdf = nx_to_gdf(drive_graph, nodes=False, edges=True)
drive_graph_gdf_buffer = buffer_directed_roads(drive_graph_gdf, 3)


In [8]:

from src.dongraphio.utils.graph_utils import convert_multidigraph_to_digraph, project_points_on_graph, \
    add_projected_points_as_nodes
from src.dongraphio.utils.tsp_solver import resolve_tsp, subgraph_by_path
from src.dongraphio.utils.matrix_utils import get_dist_matrix_for_tsp
import networkx as nx

city_crs = 32636
bus_routes = nx.DiGraph()
error_routes = []
for index, row in tqdm(devided_routes.iterrows(), total=devided_routes.shape[0], desc=f"Расчёт маршрута {index}"):
    try:
        geometry_list = row["geometry"]
        row_gdf = gpd.GeoDataFrame(data={"label": [index for _ in geometry_list], "geometry": geometry_list})
        row_gdf = row_gdf.set_crs(4326)
        row_gdf = row_gdf.to_crs(city_crs)

        projected_points = project_points_on_graph(drive_graph_gdf, drive_graph_gdf_buffer, row_gdf)

        route_nodes, graph_with_new_points = add_projected_points_as_nodes(projected_points, drive_graph)
        graph_with_new_points = convert_multidigraph_to_digraph(graph_with_new_points)
        distance_matrix, path_matrix = get_dist_matrix_for_tsp(graph_with_new_points, route_nodes)
        path = resolve_tsp(distance_matrix,5)
        route_nx = subgraph_by_path(path, path_matrix, graph_with_new_points)
        bus_routes = nx.compose(bus_routes, route_nx)

        del graph_with_new_points, projected_points, route_nodes, distance_matrix, path_matrix, path, route_nx
    except Exception:
        error_routes.append((index, row))
        pass




Расчёт маршрута 851Д.3:   0%|          | 0/847 [00:00<?, ?it/s]

,0,1
0,851Д.3,geometry [POINT (30.4441693 60.050867)] Nam...
1,546,"geometry [POINT (30.1444949 59.7893016), PO..."
2,3.1,"geometry [POINT (32.3536583 59.8943022), PO..."
3,Т-3\,geometry [POINT (30.866063829817104 59.5387...
4,2.5,"geometry [POINT (32.3536583 59.8943022), PO..."
...,...,...
61,73,geometry [POINT (28.6220888 59.3752543)] Na...
62,5.2,"geometry [POINT (32.3536583 59.8943022), PO..."
63,145.3,"geometry [POINT (30.1444949 59.7893016), PO..."
64,879Д.1,"geometry [POINT (32.3536583 59.8943022), PO..."


In [19]:
from shapely import MultiPoint

data = [{'route':obj[0],'geometry':MultiPoint(obj[1]['geometry'])} for obj in error_routes]
errors = gpd.GeoDataFrame(data=data,geometry='geometry',crs=4326)
errors.to_file('Error_routes.geojson')

In [11]:
for i in bus_routes.edges(data=True):
    i[2]['geometry'] = from_wkt(str(i[2]['geometry']))
bus_routes.graph['crs'] = city_crs
gdf_graph_edges = ox.graph_to_gdfs(nx.MultiDiGraph(bus_routes), nodes=False)
gdf_graph_edges.to_file('bus_LO.geojson')

In [10]:
for i in bus_routes.edges(data=True):
    i[2]['geometry'] = (str(i[2]['geometry']))
nx.write_graphml(bus_routes, 'bus_LO.graphml')

In [ ]:
# test = nx.DiGraph(drive_graph)
# print(test.get_edge_data(68414, 70235))
drive_graph['202466']